In [206]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
import json
import datetime


from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn.cluster import AgglomerativeClustering
#import scipy.cluster.hierarchy as sch

#import April12.ipynb
#%store -r icd_grouped 
import statsmodels.api as sm
from patsy import dmatrices 

# model selection
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

import seaborn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 

# Demographics

In [79]:
data2= pd.read_csv('ADMISSIONS.csv')
data2.head()
data2.shape # doesn't have empty rows
#data.head()
#data['SUBJECT_ID'].head(10)

(58976, 19)

In [80]:
data2.dtypes

ROW_ID                   int64
SUBJECT_ID               int64
HADM_ID                  int64
ADMITTIME               object
DISCHTIME               object
DEATHTIME               object
ADMISSION_TYPE          object
ADMISSION_LOCATION      object
DISCHARGE_LOCATION      object
INSURANCE               object
LANGUAGE                object
RELIGION                object
MARITAL_STATUS          object
ETHNICITY               object
EDREGTIME               object
EDOUTTIME               object
DIAGNOSIS               object
HOSPITAL_EXPIRE_FLAG     int64
HAS_CHARTEVENTS_DATA     int64
dtype: object

In [81]:
df = data2[['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMISSION_TYPE', 'INSURANCE', 'MARITAL_STATUS', 'ETHNICITY', 'DIAGNOSIS']]
df.head(30)




,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME
5,26,26,197661,EMERGENCY,Medicare,SINGLE,UNKNOWN/NOT SPECIFIED,V-TACH
6,27,27,134931,NEWBORN,Private,NaN,WHITE,NEWBORN
7,28,28,162569,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...
8,29,30,104557,URGENT,Medicare,MARRIED,UNKNOWN/NOT SPECIFIED,UNSTABLE ANGINA\CATH
9,30,31,128652,EMERGENCY,Medicare,MARRIED,WHITE,STATUS EPILEPTICUS


# Label Encoding

In [82]:
le = LabelEncoder()
df['ADMISSION_TYPEc']= le.fit_transform(df['ADMISSION_TYPE']) 
df.head(10)
#df.loc[df['ADMISSION_TYPEc']==4]

/Users/kathy908000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1
5,26,26,197661,EMERGENCY,Medicare,SINGLE,UNKNOWN/NOT SPECIFIED,V-TACH,1
6,27,27,134931,NEWBORN,Private,NaN,WHITE,NEWBORN,2
7,28,28,162569,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0
8,29,30,104557,URGENT,Medicare,MARRIED,UNKNOWN/NOT SPECIFIED,UNSTABLE ANGINA\CATH,3
9,30,31,128652,EMERGENCY,Medicare,MARRIED,WHITE,STATUS EPILEPTICUS,1


In [83]:
#preprocess NaN
df.fillna("NAN", inplace = True)
df.head(20)

/Users/kathy908000/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1
5,26,26,197661,EMERGENCY,Medicare,SINGLE,UNKNOWN/NOT SPECIFIED,V-TACH,1
6,27,27,134931,NEWBORN,Private,NAN,WHITE,NEWBORN,2
7,28,28,162569,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0
8,29,30,104557,URGENT,Medicare,MARRIED,UNKNOWN/NOT SPECIFIED,UNSTABLE ANGINA\CATH,3
9,30,31,128652,EMERGENCY,Medicare,MARRIED,WHITE,STATUS EPILEPTICUS,1


In [84]:
df['INSURANCEc']= le.fit_transform(df['INSURANCE']) 
df['MARITAL_STATUSc']= le.fit_transform(df['MARITAL_STATUS']) 
df['ETHNICITYc']= le.fit_transform(df['ETHNICITY']) 

df.head(30)

/Users/kathy908000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/kathy908000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/kathy908000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,INSURANCEc,MARITAL_STATUSc,ETHNICITYc
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,3,2,36
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,2,2,36
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1,2,2,36
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,3,5,36
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,3,2,36
5,26,26,197661,EMERGENCY,Medicare,SINGLE,UNKNOWN/NOT SPECIFIED,V-TACH,1,2,5,35
6,27,27,134931,NEWBORN,Private,NAN,WHITE,NEWBORN,2,3,3,36
7,28,28,162569,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,2,2,36
8,29,30,104557,URGENT,Medicare,MARRIED,UNKNOWN/NOT SPECIFIED,UNSTABLE ANGINA\CATH,3,2,2,35
9,30,31,128652,EMERGENCY,Medicare,MARRIED,WHITE,STATUS EPILEPTICUS,1,2,2,36


In [85]:
df.loc[df['INSURANCEc']==3]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,INSURANCEc,MARITAL_STATUSc,ETHNICITYc
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,3,2,36
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,3,5,36
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,3,2,36
6,27,27,134931,NEWBORN,Private,NAN,WHITE,NEWBORN,2,3,3,36
20,41,39,106266,NEWBORN,Private,NAN,UNKNOWN/NOT SPECIFIED,NEWBORN,2,3,3,35
...,...,...,...,...,...,...,...,...,...,...,...,...
58968,58591,98790,187987,EMERGENCY,Private,MARRIED,WHITE,JOINT EFFUSION,1,3,2,36
58971,58594,98800,191113,EMERGENCY,Private,SINGLE,WHITE,TRAUMA,1,3,5,36
58973,58596,98805,122631,ELECTIVE,Private,MARRIED,WHITE,RENAL CANCER/SDA,0,3,2,36
58974,58597,98813,170407,EMERGENCY,Private,MARRIED,WHITE,S/P FALL,1,3,2,36


onehotencoder = OneHotEncoder()
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 
df = np.array(columnTransformer.fit_transform(df), dtype = np.str) 


In [86]:
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(df[['ADMISSION_TYPEc']]).toarray())

enc_df.head(20)

,0,1,2,3
0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,1.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0
9,0.0,1.0,0.0,0.0


In [87]:
# merge with main df bridge_df on key values
df = df.join(enc_df)
df.head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,INSURANCEc,MARITAL_STATUSc,ETHNICITYc,0,1,2,3
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,3,2,36,0.0,1.0,0.0,0.0
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,2,2,36,1.0,0.0,0.0,0.0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1,2,2,36,0.0,1.0,0.0,0.0
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,3,5,36,0.0,1.0,0.0,0.0
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,3,2,36,0.0,1.0,0.0,0.0
5,26,26,197661,EMERGENCY,Medicare,SINGLE,UNKNOWN/NOT SPECIFIED,V-TACH,1,2,5,35,0.0,1.0,0.0,0.0
6,27,27,134931,NEWBORN,Private,NAN,WHITE,NEWBORN,2,3,3,36,0.0,0.0,1.0,0.0
7,28,28,162569,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,2,2,36,1.0,0.0,0.0,0.0
8,29,30,104557,URGENT,Medicare,MARRIED,UNKNOWN/NOT SPECIFIED,UNSTABLE ANGINA\CATH,3,2,2,35,0.0,0.0,0.0,1.0
9,30,31,128652,EMERGENCY,Medicare,MARRIED,WHITE,STATUS EPILEPTICUS,1,2,2,36,0.0,1.0,0.0,0.0


In [88]:
#rename onehotencoded columns
df = df.rename({0: 'ELECTIVE', 1: 'EMERGENCY', 2: 'NEWBORN', 3: 'URGENT'}, axis=1)
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,INSURANCEc,MARITAL_STATUSc,ETHNICITYc,ELECTIVE,EMERGENCY,NEWBORN,URGENT
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,3,2,36,0.0,1.0,0.0,0.0
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,2,2,36,1.0,0.0,0.0,0.0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1,2,2,36,0.0,1.0,0.0,0.0
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,3,5,36,0.0,1.0,0.0,0.0
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,3,2,36,0.0,1.0,0.0,0.0


In [89]:
enc_df2 = pd.DataFrame(enc.fit_transform(df[['INSURANCEc']]).toarray())

enc_df2.head()
df = df.join(enc_df2)
df.head(10)
#rename onehotencoded columns
df = df.rename({0: 'Government', 1: 'Medicaid', 2: 'Medicare', 3: 'Private', 4: 'Self Pay'}, axis=1)
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,INSURANCEc,...,ETHNICITYc,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,3,...,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,2,...,36,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1,2,...,36,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,3,...,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,3,...,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [90]:
enc_df3 = pd.DataFrame(enc.fit_transform(df[['MARITAL_STATUSc']]).toarray())

enc_df3.head()
df = df.join(enc_df3)
df.head(10)
#rename onehotencoded columns
df = df.rename({0: 'DIVORCED', 1: 'LIFE PARTNER', 2: 'MARRIED', 3: 'NAN', 4: 'SEPARATED', 5:'SINGLE', 6: 'UNKNOWN', 7: 'WIDOWED' }, axis=1)
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,INSURANCEc,...,Private,Self Pay,DIVORCED,LIFE PARTNER,MARRIED,NAN,SEPARATED,SINGLE,UNKNOWN,WIDOWED
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [91]:
enc_df4 = pd.DataFrame(enc.fit_transform(df[['ETHNICITYc']]).toarray())

enc_df4.head()
df = df.join(enc_df4)
df.head(10)
#number columns are ethnicity 

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,INSURANCEc,...,31,32,33,34,35,36,37,38,39,40
0,21,22,165315,EMERGENCY,Private,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,22,23,152223,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,23,23,124321,EMERGENCY,Medicare,MARRIED,WHITE,BRAIN MASS,1,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,24,24,161859,EMERGENCY,Private,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,25,25,129635,EMERGENCY,Private,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,26,26,197661,EMERGENCY,Medicare,SINGLE,UNKNOWN/NOT SPECIFIED,V-TACH,1,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,27,27,134931,NEWBORN,Private,NAN,WHITE,NEWBORN,2,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,28,28,162569,ELECTIVE,Medicare,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,29,30,104557,URGENT,Medicare,MARRIED,UNKNOWN/NOT SPECIFIED,UNSTABLE ANGINA\CATH,3,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,30,31,128652,EMERGENCY,Medicare,MARRIED,WHITE,STATUS EPILEPTICUS,1,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [92]:
df = df.sort_values(by=['SUBJECT_ID'])
#mergednodup = mergednodup.sort_values(by=['SUBJECT_ID'])
df[['SUBJECT_ID', 'ELECTIVE']].head(30)

,SUBJECT_ID,ELECTIVE
211,2,0.0
212,3,0.0
213,4,0.0
214,5,0.0
215,6,1.0
216,7,0.0
217,8,0.0
218,9,0.0
219,10,0.0
220,11,0.0


# Check weather groupby works

In [93]:
df.dtypes

ROW_ID              int64
SUBJECT_ID          int64
HADM_ID             int64
ADMISSION_TYPE     object
INSURANCE          object
                   ...   
36                float64
37                float64
38                float64
39                float64
40                float64
Length: 70, dtype: object

In [94]:
#round, remove the decimal points

df = df.astype({'ELECTIVE': int, 'EMERGENCY': int, 'NEWBORN': int, 'URGENT': int  })
df = df.astype({'ELECTIVE': str, 'EMERGENCY': str, 'NEWBORN': str, 'URGENT': str })




#df['ELECTIVE'].apply(np.ceil)

#df['ELECTIVE'] = df['ELECTIVE'].round(decimals=0)
#df['ELECTIVE'] = (df['ELECTIVE'] // 1e-5) * 1e-5
#df = df.astype ({'ELECTIVE': 'int' }) 
#df['ELECTIVE'] = df['ELECTIVE'].astype(int)
#df['ELECTIVE'] = (df['ELECTIVE'] * 1e2).astype(int) / 1e2

#df.style.set_precision(1)
#df.style.format({ 'ELECTIVE': '{:,.1f}'.format, 'EMERGENCY': '{:,.3f}'.format, })

#df.round()
#df = df.astype ({'ELECTIVE': 'int32' }) 
#df = df.astype ({'ELECTIVE': str }) 
#df['ELECTIVE'].head()
df.head(20)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,ADMISSION_TYPEc,INSURANCEc,...,31,32,33,34,35,36,37,38,39,40
211,1,2,163353,NEWBORN,Private,NAN,ASIAN,NEWBORN,2,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
212,2,3,145834,EMERGENCY,Medicare,MARRIED,WHITE,HYPOTENSION,1,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
213,3,4,185777,EMERGENCY,Private,SINGLE,WHITE,"FEVER,DEHYDRATION,FAILURE TO THRIVE",1,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
214,4,5,178980,NEWBORN,Private,NAN,ASIAN,NEWBORN,2,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
215,5,6,107064,ELECTIVE,Medicare,MARRIED,WHITE,CHRONIC RENAL FAILURE/SDA,0,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
216,6,7,118037,NEWBORN,Private,NAN,WHITE,NEWBORN,2,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
217,7,8,159514,NEWBORN,Private,NAN,WHITE,NEWBORN,2,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
218,8,9,150750,EMERGENCY,Medicaid,NAN,UNKNOWN/NOT SPECIFIED,HEMORRHAGIC CVA,1,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
219,9,10,184167,NEWBORN,Medicaid,NAN,BLACK/AFRICAN AMERICAN,NEWBORN,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
220,10,11,194540,EMERGENCY,Private,MARRIED,WHITE,BRAIN MASS,1,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [95]:
cols = [0,2,3,4,5,6,7,8,9,10,11]
dfpart= df.drop(df.columns[cols],axis=1)
#dfpart = df.iloc[:, 8:]
dfpart.head()
# need subject id 

,SUBJECT_ID,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,...,31,32,33,34,35,36,37,38,39,40
211,2,0,0,1,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
212,3,0,1,0,0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
213,4,0,1,0,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
214,5,0,0,1,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
215,6,1,0,0,0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [96]:
dfpart['SUBJECT_ID']

211          2
212          3
213          4
214          5
215          6
         ...  
56435    99985
56436    99991
56437    99992
56565    99995
56566    99999
Name: SUBJECT_ID, Length: 58976, dtype: int64

In [97]:
dfpart.head()

,SUBJECT_ID,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,...,31,32,33,34,35,36,37,38,39,40
211,2,0,0,1,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
212,3,0,1,0,0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
213,4,0,1,0,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
214,5,0,0,1,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
215,6,1,0,0,0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [98]:
dfpart.iloc[:,0]

211          2
212          3
213          4
214          5
215          6
         ...  
56435    99985
56436    99991
56437    99992
56565    99995
56566    99999
Name: SUBJECT_ID, Length: 58976, dtype: int64

In [99]:
dfpart = dfpart.astype(int)
dfpart = dfpart.astype(str)
dfpart.head()


,SUBJECT_ID,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,...,31,32,33,34,35,36,37,38,39,40
211,2,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
212,3,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
213,4,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
214,5,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
215,6,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [100]:
#convert subject ID back to int 
dfpart = dfpart.astype({'SUBJECT_ID': int}) 

In [101]:
dfpart.dtypes

SUBJECT_ID       int64
ELECTIVE        object
EMERGENCY       object
NEWBORN         object
URGENT          object
Government      object
Medicaid        object
Medicare        object
Private         object
Self Pay        object
DIVORCED        object
LIFE PARTNER    object
MARRIED         object
NAN             object
SEPARATED       object
SINGLE          object
UNKNOWN         object
WIDOWED         object
0               object
1               object
2               object
3               object
4               object
5               object
6               object
7               object
8               object
9               object
10              object
11              object
12              object
13              object
14              object
15              object
16              object
17              object
18              object
19              object
20              object
21              object
22              object
23              object
24              object
25         

In [102]:
#check join 
ndf = df.groupby('SUBJECT_ID').agg({'ELECTIVE' : lambda x: ''.join(x)})
ndf.head(30)
#ndf['ELECTIVE'].head(30)


,ELECTIVE
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,1
7,0
8,0
9,0
10,0


ndf = df.groupby('SUBJECT_ID').agg({'ELECTIVE' : lambda x: ''.join(x)})
ndf.head(30)

ndf.dtypes

#ndf = ndf.astype(str)
ndf = ndf.astype ({'ELECTIVE': str }) 


ndf.dtypes

df['ELECTIVE'] = ndf['ELECTIVE']
df.head(30)

#check regex
a  = 
#ndf.loc[ndf['ELECTIVE']str.contains('1'), ELECTIVE'] = 1

print(ndf)

In [103]:
#elective = ndf['ELECTIVE']
#elective.head(20)

In [104]:
ndf.dtypes

ELECTIVE    object
dtype: object

In [105]:
#ndf = ndf.to_string()
ndf = ndf.astype({'ELECTIVE': int })
ndf= ndf.applymap(str) # whole dataframe is string

#ndf = ndf['ELECTIVE'].apply(str)

#ndf = ndf.astype({'ELECTIVE': str })



#ndf = ndf['ELECTIVE'].astype(int)

#ndf = ndf['ELECTIVE'].astype(str)
ndf.head(20)

,ELECTIVE
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,1
7,0
8,0
9,0
10,0


In [106]:
ndf.dtypes

ELECTIVE    object
dtype: object

In [107]:
#ndf = ndf[['SUBJECT_ID','ELECTIVE']]
#ndf['SUBJECT_ID']
ndf.iloc[:, 0]

SUBJECT_ID
2        0
3        0
4        0
5        0
6        1
        ..
99985    0
99991    1
99992    0
99995    1
99999    1
Name: ELECTIVE, Length: 46520, dtype: object

In [108]:
elective = ndf['ELECTIVE']
elective.head(20)

SUBJECT_ID
2      0
3      0
4      0
5      0
6      1
7      0
8      0
9      0
10     0
11     0
12     1
13     0
16     0
17    10
18     0
19     0
20     1
21     0
22     0
23    10
Name: ELECTIVE, dtype: object

In [109]:
if '0' in '1011':
    print('yes')
else:
    print('no')

yes


In [110]:
elective.iloc[13 ]

'10'

In [111]:
ndf['ELECTIVE'][13]

'0'

In [112]:

for i in range(46520):
    #row  = elective.iloc[i]
    #if  row.contains('1'):
    if '1' in elective.iloc[i]: 
        elective.iloc[i] = '1'       
    else:
        elective.iloc[i] = '0'


In [113]:
ndf['ELECTIVE'] = elective
ndf.head(30)

,ELECTIVE
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,1
7,0
8,0
9,0
10,0



for i in range(46520):
    row  = elective.iloc[i]
    #if  row.contains('1'):
    if '1' in row: 
        ndf['ELECTIVE'][i] = '1'       
    else:
        ndf['ELECTIVE'][i] = '0'


In [114]:
dfpart.iloc[:,0] 

211          2
212          3
213          4
214          5
215          6
         ...  
56435    99985
56436    99991
56437    99992
56565    99995
56566    99999
Name: SUBJECT_ID, Length: 58976, dtype: int64

# loop through each column 
#group by, dfpart.iloc[:,0]
#2nd to 17th column
arr = []
for i in range(1,18):
    col = dfpart.iloc[:,i]
    groupedcol = dfpart.groupby('SUBJECT_ID').agg({col : lambda x: ''.join(x)})
    arr.apend(groupedcol)
newdf = arr
#arr[0] is subject id 

    #newdf = dfpart

ELECTIVE        object
EMERGENCY       object
NEWBORN         object
URGENT          object
Government      object
Medicaid        object
Medicare        object
Private         object
Self Pay        object
DIVORCED        object
LIFE PARTNER    object
MARRIED         object
NAN             object
SEPARATED       object
SINGLE          object
UNKNOWN         object
WIDOWED         object

In [115]:
dfpart.head(20)

,SUBJECT_ID,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,...,31,32,33,34,35,36,37,38,39,40
211,2,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
212,3,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
213,4,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
214,5,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
215,6,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
216,7,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
217,8,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
218,9,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
219,10,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
220,11,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [116]:
dfpart['SUBJECT_ID'].head()

211    2
212    3
213    4
214    5
215    6
Name: SUBJECT_ID, dtype: int64

In [117]:
dfpart.dtypes

SUBJECT_ID       int64
ELECTIVE        object
EMERGENCY       object
NEWBORN         object
URGENT          object
Government      object
Medicaid        object
Medicare        object
Private         object
Self Pay        object
DIVORCED        object
LIFE PARTNER    object
MARRIED         object
NAN             object
SEPARATED       object
SINGLE          object
UNKNOWN         object
WIDOWED         object
0               object
1               object
2               object
3               object
4               object
5               object
6               object
7               object
8               object
9               object
10              object
11              object
12              object
13              object
14              object
15              object
16              object
17              object
18              object
19              object
20              object
21              object
22              object
23              object
24              object
25         

In [118]:
ndf2 = dfpart.groupby('SUBJECT_ID').agg({'EMERGENCY' : lambda x: ''.join(x)})
ndf3 = dfpart.groupby('SUBJECT_ID').agg({'NEWBORN' : lambda x: ''.join(x)})
ndf4 = dfpart.groupby('SUBJECT_ID').agg({'URGENT' : lambda x: ''.join(x)})

ndf5 = dfpart.groupby('SUBJECT_ID').agg({'Government' : lambda x: ''.join(x)})
ndf6 = dfpart.groupby('SUBJECT_ID').agg({'Medicaid' : lambda x: ''.join(x)})
ndf7 = dfpart.groupby('SUBJECT_ID').agg({'Medicare' : lambda x: ''.join(x)})
ndf8 = dfpart.groupby('SUBJECT_ID').agg({'Private' : lambda x: ''.join(x)})
ndf9 = dfpart.groupby('SUBJECT_ID').agg({'Self Pay' : lambda x: ''.join(x)})



ndf10 = dfpart.groupby('SUBJECT_ID').agg({'DIVORCED' : lambda x: ''.join(x)})

ndf11 = dfpart.groupby('SUBJECT_ID').agg({'LIFE PARTNER' : lambda x: ''.join(x)})
ndf12 = dfpart.groupby('SUBJECT_ID').agg({'MARRIED' : lambda x: ''.join(x)})
ndf13 = dfpart.groupby('SUBJECT_ID').agg({'NAN' : lambda x: ''.join(x)})
ndf14 = dfpart.groupby('SUBJECT_ID').agg({'SEPARATED' : lambda x: ''.join(x)})
ndf15 = dfpart.groupby('SUBJECT_ID').agg({'SINGLE' : lambda x: ''.join(x)})
ndf16 = dfpart.groupby('SUBJECT_ID').agg({'UNKNOWN' : lambda x: ''.join(x)})
ndf17 = dfpart.groupby('SUBJECT_ID').agg({'WIDOWED' : lambda x: ''.join(x)})



In [119]:
#ndf17 = ndf17.sort_values(by=['SUBJECT_ID'])
ndf4.head(20)

,URGENT
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
10,0


In [120]:
#ndf = ndf.astype({'ELECTIVE': int })
#ndf= ndf.applymap(str)

In [121]:

for i in range(46520):
    #row  = elective.iloc[i]
    #if  row.contains('1'):
    if '1' in ndf['ELECTIVE'].iloc[i]: 
        ndf['ELECTIVE'].iloc[i] = '1'       
    else:
        ndf['ELECTIVE'].iloc[i] = '0'

In [122]:
ndf.head(10)

,ELECTIVE
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,1
7,0
8,0
9,0
10,0


In [123]:
for i in range(46520):
    #row  = elective.iloc[i]
    #if  row.contains('1'):
    if '1' in ndf['ELECTIVE'].iloc[i]: 
        ndf['ELECTIVE'].iloc[i] = '1'       
    else:
        ndf['ELECTIVE'].iloc[i] = '0'

In [124]:
for i in range(46520):
    if '1' in ndf2['EMERGENCY'].iloc[i]: 
        ndf2['EMERGENCY'].iloc[i] = '1'       
    else:
        ndf2['EMERGENCY'].iloc[i] = '0'
ndf2.head()

,EMERGENCY
SUBJECT_ID,
2,0
3,1
4,1
5,0
6,0


In [125]:
for i in range(46520):
    if '1' in ndf3['NEWBORN'].iloc[i]: 
        ndf3['NEWBORN'].iloc[i] = '1'       
    else:
        ndf3['NEWBORN'].iloc[i] = '0'
ndf3.head()

,NEWBORN
SUBJECT_ID,
2,1
3,0
4,0
5,1
6,0


In [126]:
for i in range(46520):
    if '1' in ndf4['URGENT'].iloc[i]: 
        ndf4['URGENT'].iloc[i] = '1'       
    else:
        ndf4['URGENT'].iloc[i] = '0'
ndf4.head()

,URGENT
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,0


In [127]:
for i in range(46520):
    if '1' in ndf5['Government'].iloc[i]: 
        ndf5['Government'].iloc[i] = '1'       
    else:
        ndf5['Government'].iloc[i] = '0'
ndf5.head()

,Government
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,0


In [128]:
for i in range(46520):
    if '1' in ndf6['Medicaid'].iloc[i]: 
        ndf6['Medicaid'].iloc[i] = '1'       
    else:
        ndf6['Medicaid'].iloc[i] = '0'
ndf6.head()

,Medicaid
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,0


In [129]:
for i in range(46520):
    if '1' in ndf7['Medicare'].iloc[i]: 
        ndf7['Medicare'].iloc[i] = '1'       
    else:
        ndf7['Medicare'].iloc[i] = '0'
ndf7.head()

,Medicare
SUBJECT_ID,
2,0
3,1
4,0
5,0
6,1


In [130]:
for i in range(46520):
    if '1' in ndf8['Private'].iloc[i]: 
        ndf8['Private'].iloc[i] = '1'       
    else:
        ndf8['Private'].iloc[i] = '0'
ndf8.head()

,Private
SUBJECT_ID,
2,1
3,0
4,1
5,1
6,0


In [131]:
for i in range(46520):
    if '1' in ndf9['Self Pay'].iloc[i]: 
        ndf9['Self Pay'].iloc[i] = '1'       
    else:
        ndf9['Self Pay'].iloc[i] = '0'
ndf9.head()

,Self Pay
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,0


In [132]:
for i in range(46520):
    if '1' in ndf10['DIVORCED'].iloc[i]: 
        ndf10['DIVORCED'].iloc[i] = '1'       
    else:
        ndf10['DIVORCED'].iloc[i] = '0'
ndf10.head()

,DIVORCED
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,0


In [133]:
for i in range(46520):
    if '1' in ndf11['LIFE PARTNER'].iloc[i]: 
        ndf11['LIFE PARTNER'].iloc[i] = '1'       
    else:
        ndf11['LIFE PARTNER'].iloc[i] = '0'
ndf11.head()

,LIFE PARTNER
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,0


In [134]:
for i in range(46520):
    if '1' in ndf12['MARRIED'].iloc[i]: 
        ndf12['MARRIED'].iloc[i] = '1'       
    else:
        ndf12['MARRIED'].iloc[i] = '0'
ndf12.head()

,MARRIED
SUBJECT_ID,
2,0
3,1
4,0
5,0
6,1


In [135]:
for i in range(46520):
    if '1' in ndf13['NAN'].iloc[i]: 
        ndf13['NAN'].iloc[i] = '1'       
    else:
        ndf13['NAN'].iloc[i] = '0'
ndf13.head()

,NAN
SUBJECT_ID,
2,1
3,0
4,0
5,1
6,0


In [136]:
for i in range(46520):
    if '1' in ndf14['SEPARATED'].iloc[i]: 
        ndf14['SEPARATED'].iloc[i] = '1'       
    else:
        ndf14['SEPARATED'].iloc[i] = '0'
ndf14.head()

,SEPARATED
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,0


In [137]:
for i in range(46520):
    if '1' in ndf15['SINGLE'].iloc[i]: 
        ndf15['SINGLE'].iloc[i] = '1'       
    else:
        ndf15['SINGLE'].iloc[i] = '0'
ndf15.head()

,SINGLE
SUBJECT_ID,
2,0
3,0
4,1
5,0
6,0


In [138]:
for i in range(46520):
    if '1' in ndf16['UNKNOWN'].iloc[i]: 
        ndf16['UNKNOWN'].iloc[i] = '1'       
    else:
        ndf16['UNKNOWN'].iloc[i] = '0'
ndf16.head()

,UNKNOWN
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,0


In [139]:
for i in range(46520):
    if '1' in ndf17['WIDOWED'].iloc[i]: 
        ndf17['WIDOWED'].iloc[i] = '1'       
    else:
        ndf17['WIDOWED'].iloc[i] = '0'
ndf17.head()

,WIDOWED
SUBJECT_ID,
2,0
3,0
4,0
5,0
6,0


# Inner join onehotcoding dataframe

In [140]:
new = pd.merge(ndf, ndf2, how='inner', on='SUBJECT_ID' )
new.head()

,ELECTIVE,EMERGENCY
SUBJECT_ID,,
2,0,0
3,0,1
4,0,1
5,0,0
6,1,0


In [141]:
new = pd.merge(new, ndf3, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf4, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf5, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf6, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf7, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf8, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf9, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf10, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf11, how='inner', on='SUBJECT_ID' )

new = pd.merge(new, ndf12, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf13, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf14, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf15, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf16, how='inner', on='SUBJECT_ID' )
new = pd.merge(new, ndf17, how='inner', on='SUBJECT_ID' )
new.head(20)


,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,DIVORCED,LIFE PARTNER,MARRIED,NAN,SEPARATED,SINGLE,UNKNOWN,WIDOWED
SUBJECT_ID,,,,,,,,,,,,,,,,,
2,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
5,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
6,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
7,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
8,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
9,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
10,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0


#ndf['SUBJECT_ID']
data3 = dfpart.drop_duplicates(subset='SUBJECT_ID', keep='first', inplace=False)
data3['SUBJECT_ID']

In [142]:

#newdf = pd.DataFrame({ndf.iloc[:, 0] ,ndf[:, 1], ndf2[:, 1] })
#newdf.head()

In [143]:
#newdf= dfpart.copy()
#newdf.head(20)
#different number of rows newdf row = patient, dfpart row = admission
#or innerjoinr with subjecct ID? 

In [144]:
#ndf.iloc[:, 1]

In [145]:
#ndf2['SUBJECT_ID']

In [146]:
#newdf['SUBJECT_ID'] = ndf['SUBJECT_ID']
#newdf['SUBJECT_ID'] = ndf.iloc[:, 0]
#newdf['ELECTIVE'] = ndf.iloc[:, 1]




# Ethnicity 

In [147]:
edf = data2.drop_duplicates(subset='SUBJECT_ID', keep='first', inplace=False)

edf.shape
edf.head()



,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,V-TACH,0,1


In [148]:
edf = edf[['SUBJECT_ID','ETHNICITY' ]]
edf.head()

,SUBJECT_ID,ETHNICITY
0,22,WHITE
1,23,WHITE
3,24,WHITE
4,25,WHITE
5,26,UNKNOWN/NOT SPECIFIED


In [149]:
le = LabelEncoder() 
edf['ETHNICITYc']= le.fit_transform(edf['ETHNICITY']) 

edf.head()

,SUBJECT_ID,ETHNICITY,ETHNICITYc
0,22,WHITE,36
1,23,WHITE,36
3,24,WHITE,36
4,25,WHITE,36
5,26,UNKNOWN/NOT SPECIFIED,35


In [150]:
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df2 = pd.DataFrame(enc.fit_transform(edf[['ETHNICITYc']]).toarray())

enc_df2.head(30)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [151]:
#remove decimals
enc_df2 = enc_df2.astype(int)
enc_df2 = enc_df2.astype(str)
enc_df2.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [152]:
# add ethnicity to new
#new = new.iloc[:, :18]

new = new.join(enc_df2)
new.head()

,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,DIVORCED,...,31,32,33,34,35,36,37,38,39,40
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
2,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [153]:
new.isnull().values.any()

True

In [154]:
new.shape

(46520, 58)

In [155]:
new2 = new.dropna()
new2.shape

(32504, 58)

# Gender

In [156]:
grouped = pd.read_csv('icd_grouped2.csv')
grouped.shape

/Users/kathy908000/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,2,3,4,6,8,9,12,13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1048575, 16)

In [157]:
gdata = grouped.drop_duplicates(subset='SUBJECT_ID', keep='first', inplace=False)
gdata.shape
gdata.head()

,ROW_ID_x,ICD9_CODE,SHORT_TITLE_x,LONG_TITLE_x,group,group2,lv.1_code,lv.2_code,lv.1_group,lv.2_group,SUBJECT_ID,HADM_ID,GENDER,DOB,DOD,DOD_HOSP
0,241.0,30,Salmonella enteritis,Salmonella gastroenteritis,0,1.0,001-009,3.0,INTESTINAL INFECTIOUS DISEASES,Other salmonella infections,27367.0,136441.0,F,5/26/21 0:00,1.0,9/29/02 0:00
1,241.0,30,Salmonella enteritis,Salmonella gastroenteritis,0,1.0,001-009,3.0,INTESTINAL INFECTIOUS DISEASES,Other salmonella infections,65733.0,168926.0,M,1/17/95 0:00,1.0,1/5/67 0:00
2,242.0,31,Salmonella septicemia,Salmonella septicemia,0,1.0,001-009,3.0,INTESTINAL INFECTIOUS DISEASES,Other salmonella infections,69141.0,136542.0,M,2/25/19 0:00,0.0,NaN
3,249.0,38,Salmonella infection NEC,Other specified salmonella infections,0,1.0,001-009,3.0,INTESTINAL INFECTIOUS DISEASES,Other salmonella infections,44437.0,181557.0,F,12/28/43 0:00,0.0,NaN
4,249.0,38,Salmonella infection NEC,Other specified salmonella infections,0,1.0,001-009,3.0,INTESTINAL INFECTIOUS DISEASES,Other salmonella infections,83908.0,180356.0,M,2/13/49 0:00,0.0,NaN


In [158]:
#genderdf = gdata['GENDER'] 
#genderdf.head()

In [159]:
#gdata['GENDER'] .fillna("NAN", inplace = True)

In [160]:
#gdata['GENDERc']= le.fit_transform(gdata['GENDER'])
#gdata.head(10)

In [161]:
#gdata.loc[gdata['GENDERc']==2]
#row 634709 has every call NA drop

In [162]:
gdata.shape

(46518, 16)

In [163]:
gdata = gdata.drop(634709)

In [164]:
gdata.shape
#d=gdata.dropna()
#d.shape

(46517, 16)

In [165]:
#gdata.loc[gdata['GENDERc']==2]

In [166]:
gdata['GENDER'] = gdata['GENDER'].map({'F':0, 'M': 1})
gdata.head()

,ROW_ID_x,ICD9_CODE,SHORT_TITLE_x,LONG_TITLE_x,group,group2,lv.1_code,lv.2_code,lv.1_group,lv.2_group,SUBJECT_ID,HADM_ID,GENDER,DOB,DOD,DOD_HOSP
0,241.0,30,Salmonella enteritis,Salmonella gastroenteritis,0,1.0,001-009,3.0,INTESTINAL INFECTIOUS DISEASES,Other salmonella infections,27367.0,136441.0,0,5/26/21 0:00,1.0,9/29/02 0:00
1,241.0,30,Salmonella enteritis,Salmonella gastroenteritis,0,1.0,001-009,3.0,INTESTINAL INFECTIOUS DISEASES,Other salmonella infections,65733.0,168926.0,1,1/17/95 0:00,1.0,1/5/67 0:00
2,242.0,31,Salmonella septicemia,Salmonella septicemia,0,1.0,001-009,3.0,INTESTINAL INFECTIOUS DISEASES,Other salmonella infections,69141.0,136542.0,1,2/25/19 0:00,0.0,NaN
3,249.0,38,Salmonella infection NEC,Other specified salmonella infections,0,1.0,001-009,3.0,INTESTINAL INFECTIOUS DISEASES,Other salmonella infections,44437.0,181557.0,0,12/28/43 0:00,0.0,NaN
4,249.0,38,Salmonella infection NEC,Other specified salmonella infections,0,1.0,001-009,3.0,INTESTINAL INFECTIOUS DISEASES,Other salmonella infections,83908.0,180356.0,1,2/13/49 0:00,0.0,NaN


In [167]:
gdata = gdata.sort_values(by=['SUBJECT_ID'])
gdata.head()

,ROW_ID_x,ICD9_CODE,SHORT_TITLE_x,LONG_TITLE_x,group,group2,lv.1_code,lv.2_code,lv.1_group,lv.2_group,SUBJECT_ID,HADM_ID,GENDER,DOB,DOD,DOD_HOSP
562539,12202.0,V053,Need prphyl vc vrl hepat,Need for prophylactic vaccination and inoculat...,V0,18.0,V01-V06,1005.0,PERSONS WITH POTENTIAL HEALTHHAZARDS RELATED T...,Need for prophylactic vaccination and inoculat...,2.0,163353.0,1,7/17/38 0:00,0.0,NaN
4473,660.0,389,Septicemia NOS,Unspecified septicemia,3,1.0,030-041,38.0,OTHER BACTERIAL DISEASES,Septicemia,3.0,145834.0,1,4/11/25 0:00,1.0,NaN
8456,493.0,4111,Mth sus Stph aur els/NOS,Methicillin susceptible Staphylococcus aureus ...,4,1.0,030-041,41.0,OTHER BACTERIAL DISEASES,Bacterial infection in conditions classified e...,4.0,185777.0,0,5/12/43 0:00,0.0,NaN
562540,12202.0,V053,Need prphyl vc vrl hepat,Need for prophylactic vaccination and inoculat...,V0,18.0,V01-V06,1005.0,PERSONS WITH POTENTIAL HEALTHHAZARDS RELATED T...,Need for prophylactic vaccination and inoculat...,5.0,178980.0,1,2/2/03 0:00,0.0,NaN
79722,2777.0,2753,Dis phosphorus metabol,Disorders of phosphorus metabolism,27,3.0,270-279,275.0,OTHER METABOLIC AND IMMUNITY DISORDERS,Disorders of mineral metabolism,6.0,107064.0,0,6/21/09 0:00,0.0,NaN


In [168]:
gdf = gdata[['SUBJECT_ID', 'GENDER']]
gdf.head()

,SUBJECT_ID,GENDER
562539,2.0,1
4473,3.0,1
8456,4.0,0
562540,5.0,1
79722,6.0,0


In [169]:
gdf.isnull().values.any()

False

In [170]:
gdf = gdf.astype(int)
gdf = gdf.astype(str)
gdf = gdf.astype({'SUBJECT_ID': int})
gdf.head()
## na values 

,SUBJECT_ID,GENDER
562539,2,1
4473,3,1
8456,4,0
562540,5,1
79722,6,0


In [171]:
gdf.dtypes

SUBJECT_ID     int64
GENDER        object
dtype: object

In [172]:
new = pd.merge(new, gdf, how='inner', on='SUBJECT_ID' )
new.head()

,SUBJECT_ID,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,...,32,33,34,35,36,37,38,39,40,GENDER
0,2,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
1,3,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
2,4,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,5,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
4,6,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


# Add Diagnosis Matrix

In [173]:
# diagnosis not grouped
didf= pd.read_csv('all_diag_matrix.csv')
didf.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,DIAGNOSES,...,72991,80119,2140,V6107,1838,17341,2982,9994,5644,nan
0,0,1,2,M,2138-07-17 00:00:00,NaN,NaN,NaN,0,"['Single liveborn, born in hospital, delivered...",...,0,0,0,0,0,0,0,0,0,NaN
1,1,2,3,M,2025-04-11 00:00:00,2102-06-14 00:00:00,NaN,2102-06-14 00:00:00,1,"['Unspecified protein-calorie malnutrition', '...",...,0,0,0,0,0,0,0,0,0,NaN
2,2,3,4,F,2143-05-12 00:00:00,NaN,NaN,NaN,0,['Antiviral drugs causing adverse effects in t...,...,0,0,0,0,0,0,0,0,0,NaN
3,3,4,5,M,2103-02-02 00:00:00,NaN,NaN,NaN,0,['Need for prophylactic vaccination and inocul...,...,0,0,0,0,0,0,0,0,0,NaN
4,4,5,6,F,2109-06-21 00:00:00,NaN,NaN,NaN,0,"['Hypertensive chronic kidney disease, unspeci...",...,0,0,0,0,0,0,0,0,0,NaN


In [174]:
didf. dtypes

Unnamed: 0      int64
ROW_ID          int64
SUBJECT_ID      int64
GENDER         object
DOB            object
               ...   
17341           int64
2982            int64
9994            int64
5644            int64
nan           float64
Length: 6999, dtype: object

In [175]:
didf.iloc[:, 2] # subject id

0            2
1            3
2            4
3            5
4            6
         ...  
46515    99985
46516    99991
46517    99992
46518    99995
46519    99999
Name: SUBJECT_ID, Length: 46520, dtype: int64

In [176]:
col = [0,1,3,4,5,6,7,8,9, 10, 11, 12]
#col = [0,1,3,4,5,6,7,8,9,10]
didf2= didf.drop(didf.columns[col],axis=1)

In [177]:
didf2.head()

,SUBJECT_ID,00845,0380,03811,0382,0383,03842,03849,0388,0389,...,72991,80119,2140,V6107,1838,17341,2982,9994,5644,nan
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [178]:
didf2['SUBJECT_ID']

0            2
1            3
2            4
3            5
4            6
         ...  
46515    99985
46516    99991
46517    99992
46518    99995
46519    99999
Name: SUBJECT_ID, Length: 46520, dtype: int64

In [179]:
#n = didf2['nan']
#didf2.iloc[:, -1]
col2= [-1]

In [180]:
didf2= didf2.drop(didf.columns[col2],axis=1)
#didf= didf.drop( didf.columns[-1],axis=1)
didf2.head()

,SUBJECT_ID,00845,0380,03811,0382,0383,03842,03849,0388,0389,...,61882,72991,80119,2140,V6107,1838,17341,2982,9994,5644
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [181]:

didf2.shape

(46520, 6986)

In [182]:
didf2.isnull().values.any()

False

In [183]:
matrix = pd.merge(new, didf2, how='inner', on='SUBJECT_ID' )
matrix.head()

,SUBJECT_ID,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,...,61882,72991,80119,2140,V6107,1838,17341,2982,9994,5644
0,2,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,6,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [184]:
#matrix.to_csv(r'/Users/kathy908000/github/Health-Care-Research/matrix.csv', index = False)

# Add ICU stay length / Hopistalization length

In [185]:
ldf= pd.read_csv('TRANSFERS.csv')
icudf = pd.read_csv('ICUSTAYS.csv')

ldf.head(20)
icudf.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [186]:
#ldf.drop_duplicates(subset='SUBJECT_ID', keep='first', inplace=False) #46520 rows, patients

In [187]:
#diff careunit
ldf.drop_duplicates(subset='CURR_CAREUNIT', keep='first', inplace=False)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,EVENTTYPE,PREV_CAREUNIT,CURR_CAREUNIT,PREV_WARDID,CURR_WARDID,INTIME,OUTTIME,LOS
0,657,111,192123,254245.0,carevue,transfer,CCU,MICU,7.0,23.0,2142-04-29 15:27:11,2142-05-04 20:38:33,125.19
1,658,111,192123,NaN,carevue,transfer,MICU,NaN,23.0,45.0,2142-05-04 20:38:33,2142-05-05 11:46:32,15.13
18,675,113,109976,224661.0,carevue,admit,NaN,TSICU,NaN,23.0,2140-12-12 22:48:08,2140-12-13 17:42:46,18.91
22,679,114,178393,258626.0,carevue,transfer,NaN,CSRU,27.0,23.0,2146-08-29 17:59:00,2146-08-31 13:30:00,43.52
26,683,115,114585,232514.0,carevue,transfer,NaN,SICU,54.0,57.0,2194-10-17 13:22:28,2194-10-24 16:26:47,171.07
30,687,116,127203,208829.0,carevue,admit,NaN,NICU,NaN,56.0,2116-10-22 21:29:32,2116-10-22 22:38:33,1.15
32,689,116,127203,NaN,carevue,transfer,NICU,NWARD,56.0,5.0,2116-10-23 19:43:32,2116-10-26 19:50:01,72.11
62,719,123,195632,227264.0,carevue,admit,NaN,CCU,NaN,57.0,2161-10-29 02:27:41,2161-10-30 14:19:12,35.86


In [188]:
ldf = ldf.sort_values(by=['SUBJECT_ID'])
ldf.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,EVENTTYPE,PREV_CAREUNIT,CURR_CAREUNIT,PREV_WARDID,CURR_WARDID,INTIME,OUTTIME,LOS
320,1,2,163353,NaN,carevue,admit,NaN,NWARD,NaN,5.0,2138-07-17 19:23:42,2138-07-17 19:39:48,0.27
322,3,2,163353,243653.0,carevue,transfer,NWARD,NICU,43.0,56.0,2138-07-17 21:20:07,2138-07-17 23:32:21,2.20
323,4,2,163353,NaN,carevue,transfer,NICU,NWARD,56.0,43.0,2138-07-17 23:32:21,2138-07-21 15:50:00,88.29
324,5,2,163353,NaN,carevue,discharge,NWARD,NaN,43.0,NaN,2138-07-21 15:50:00,NaN,NaN
321,2,2,163353,NaN,carevue,transfer,NWARD,NWARD,5.0,43.0,2138-07-17 19:39:48,2138-07-17 21:20:07,1.67


In [189]:
ldf2= ldf.groupby(['SUBJECT_ID']).sum()
#ldf2.head()

icudf2 = icudf.groupby(['SUBJECT_ID']).sum()
icudf2.head()

,ROW_ID,HADM_ID,ICUSTAY_ID,FIRST_WARDID,LAST_WARDID,LOS
SUBJECT_ID,,,,,,
2,1,163353,243653,56,56,0.0918
3,2,145834,211552,12,12,6.0646
4,3,185777,294638,52,52,1.6785
5,4,178980,214757,56,56,0.0844
6,5,107064,228232,33,33,3.6729


In [190]:
icudf2.dtypes

ROW_ID            int64
HADM_ID           int64
ICUSTAY_ID        int64
FIRST_WARDID      int64
LAST_WARDID       int64
LOS             float64
dtype: object

In [191]:
#d3 = icudf2[['SUBJECT_ID', 'LOS']]
col3 = [0,1,2,3,4]

d3= icudf2.drop(icudf2.columns[col3],axis=1)
d3.head()


,LOS
SUBJECT_ID,
2,0.0918
3,6.0646
4,1.6785
5,0.0844
6,3.6729


# Number of Hopistalization 

In [192]:
d= pd.read_csv('ADMISSIONS.csv')
d.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [193]:
d['HADM_ID'].is_unique 

True

In [194]:
d = d.sort_values(by=['SUBJECT_ID'])
d.head(20)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
211,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
212,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1
213,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,2191-03-15 13:10:00,2191-03-16 01:10:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1
214,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
215,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,CHRONIC RENAL FAILURE/SDA,0,1
216,6,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1
217,7,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1
218,8,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicaid,NaN,UNOBTAINABLE,NaN,UNKNOWN/NOT SPECIFIED,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,1
219,9,10,184167,2103-06-28 11:36:00,2103-07-06 12:10:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Medicaid,NaN,UNOBTAINABLE,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,NEWBORN,0,1
220,10,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,NaN,OTHER,MARRIED,WHITE,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,1


In [195]:
d2= d.groupby(['SUBJECT_ID']).count() # number of rows = number of hospitalization/admission per patient
d2.head(20)

,ROW_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
SUBJECT_ID,,,,,,,,,,,,,,,,,,
2,1,1,1,1,0,1,1,1,1,0,1,0,1,0,0,1,1,1
3,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1
4,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1
5,1,1,1,1,0,1,1,1,1,0,1,0,1,0,0,1,1,1
6,1,1,1,1,0,1,1,1,1,1,1,1,1,0,0,1,1,1
7,1,1,1,1,0,1,1,1,1,0,1,0,1,0,0,1,1,1
8,1,1,1,1,0,1,1,1,1,0,1,0,1,0,0,1,1,1
9,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1
10,1,1,1,1,0,1,1,1,1,0,1,0,1,0,0,1,1,1


In [196]:
d2.loc[d2['ROW_ID']==10]

,ROW_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
SUBJECT_ID,,,,,,,,,,,,,,,,,,
8799,10,10,10,10,0,10,10,10,10,7,10,10,10,9,9,10,10,10
16200,10,10,10,10,1,10,10,10,10,6,10,10,10,10,10,10,10,10
16994,10,10,10,10,0,10,10,10,10,0,10,10,10,10,10,10,10,10
20124,10,10,10,10,1,10,10,10,10,10,10,10,10,9,9,10,10,10
24573,10,10,10,10,0,10,10,10,10,10,10,10,10,9,9,10,10,10
24995,10,10,10,10,0,10,10,10,10,9,10,10,10,10,10,10,10,10
25941,10,10,10,10,0,10,10,10,10,9,10,10,10,10,10,10,10,10
27362,10,10,10,10,0,10,10,10,10,10,10,10,10,9,9,10,10,10
29137,10,10,10,10,1,10,10,10,10,10,10,10,10,3,3,10,10,10


In [197]:
d2.dtypes

ROW_ID                  int64
HADM_ID                 int64
ADMITTIME               int64
DISCHTIME               int64
DEATHTIME               int64
ADMISSION_TYPE          int64
ADMISSION_LOCATION      int64
DISCHARGE_LOCATION      int64
INSURANCE               int64
LANGUAGE                int64
RELIGION                int64
MARITAL_STATUS          int64
ETHNICITY               int64
EDREGTIME               int64
EDOUTTIME               int64
DIAGNOSIS               int64
HOSPITAL_EXPIRE_FLAG    int64
HAS_CHARTEVENTS_DATA    int64
dtype: object

In [198]:
col4= [1,2, 3,4,5,6,7,8,9, 10, 11, 12, 13,14,15,16,17]
#col = [0,1,3,4,5,6,7,8,9,10]
d4= d2.drop(d2.columns[col4],axis=1)
d4.head()

,ROW_ID
SUBJECT_ID,
2,1
3,1
4,1
5,1
6,1


In [199]:
d4 = d4.rename({'ROW_ID': 'HOSPITALIZATION'}, axis=1)
d4.head()

,HOSPITALIZATION
SUBJECT_ID,
2,1
3,1
4,1
5,1
6,1


In [200]:
yval= pd.merge(d3, d4, how='inner', on='SUBJECT_ID')
yval.head()

,LOS,HOSPITALIZATION
SUBJECT_ID,,
2,0.0918,1
3,6.0646,1
4,1.6785,1
5,0.0844,1
6,3.6729,1


In [201]:
matrix2 = pd.merge(yval, new, how='inner', on='SUBJECT_ID')

In [202]:
matrix2.head()

,SUBJECT_ID,LOS,HOSPITALIZATION,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,...,32,33,34,35,36,37,38,39,40,GENDER
0,2,0.0918,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,3,6.0646,1,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
2,4,1.6785,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,5,0.0844,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,6,3.6729,1,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


# Join Diabetes Matrix

In [207]:
diadf= pd.read_csv('diabetes_binary.csv')
diadf.head()

,Unnamed: 0,SUBJECT_ID,GENDER,DIAGNOSES,ICD9_CODE,SEQ_NUM,HADM_ID,001,002,003,...,M9970,V85,a,V88,V87,V90,E000,E001,E016,V86
0,0,13,F,"['Pure hypercholesterolemia', 'Unspecified ess...","['2720', '4019', '4111', '41401', '25000']","[5.0, 4.0, 2.0, 1.0, 3.0]","[143045, 143045, 143045, 143045, 143045]",0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,18,M,"['Pure hypercholesterolemia', 'Right bundle br...","['2720', '4264', '4019', '78321', '78057', '47...","[12.0, 11.0, 10.0, 9.0, 8.0, 7.0, 13.0, 4.0, 3...","[188822, 188822, 188822, 188822, 188822, 18882...",0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,20,F,['Coronary atherosclerosis of native coronary ...,"['41401', '4111', '25000', '2724', '4019']","[1.0, 2.0, 3.0, 4.0, 5.0]","[157681, 157681, 157681, 157681, 157681]",0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,21,M,"['Abscess of liver', 'Intestinal infection due...","['5720', '00845', '70709', '6823', '5119', '99...","[10.0, 9.0, 5.0, 7.0, 6.0, 11.0, 8.0, 12.0, 4....","[111970, 111970, 111970, 111970, 111970, 11197...",0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,24,M,['Diabetes mellitus without mention of complic...,"['25000', '41401', '41041', '53081']","[4.0, 2.0, 1.0, 3.0]","[161859, 161859, 161859, 161859]",0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [208]:
diadf.shape

(10318, 3268)

In [209]:
diadf.dtypes

Unnamed: 0      int64
SUBJECT_ID      int64
GENDER         object
DIAGNOSES      object
ICD9_CODE      object
               ...   
V90           float64
E000          float64
E001          float64
E016          float64
V86           float64
Length: 3268, dtype: object

In [210]:
list(diadf.columns) 

['Unnamed: 0',
 'SUBJECT_ID',
 'GENDER',
 'DIAGNOSES',
 'ICD9_CODE',
 'SEQ_NUM',
 'HADM_ID',
 '001',
 '002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '008',
 '009',
 '010',
 '011',
 '012',
 '013',
 '014',
 '015',
 '016',
 '017',
 '018',
 '019',
 '020',
 '021',
 '022',
 '023',
 '024',
 '025',
 '026',
 '027',
 '028',
 '029',
 '030',
 '031',
 '032',
 '033',
 '034',
 '035',
 '036',
 '037',
 '038',
 '039',
 '040',
 '041',
 '042',
 '043',
 '044',
 '045',
 '046',
 '047',
 '048',
 '049',
 '050',
 '051',
 '052',
 '053',
 '054',
 '055',
 '056',
 '057',
 '058',
 '059',
 '060',
 '061',
 '062',
 '063',
 '064',
 '065',
 '066',
 '067',
 '068',
 '069',
 '070',
 '071',
 '072',
 '073',
 '074',
 '075',
 '076',
 '077',
 '078',
 '079',
 '080',
 '081',
 '082',
 '083',
 '084',
 '085',
 '086',
 '087',
 '088',
 '089',
 '090',
 '091',
 '092',
 '093',
 '094',
 '095',
 '096',
 '097',
 '098',
 '099',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '1

In [213]:
for col in diadf.columns: 
    print(col) 

Unnamed: 0
SUBJECT_ID
GENDER
DIAGNOSES
ICD9_CODE
SEQ_NUM
HADM_ID
001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049
050
051
052
053
054
055
056
057
058
059
060
061
062
063
064
065
066
067
068
069
070
071
072
073
074
075
076
077
078
079
080
081
082
083
084
085
086
087
088
089
090
091
092
093
094
095
096
097
098
099
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234

M9128
M9129
M9130
M9131
M9132
M9133
M9134
M9135
M9136
M9137
M9138
M9139
M9140
M9141
M9142
M9143
M9144
M9145
M9146
M9147
M9148
M9149
M9150
M9151
M9152
M9153
M9154
M9155
M9156
M9157
M9158
M9159
M9160
M9161
M9162
M9163
M9164
M9165
M9166
M9167
M9168
M9169
M9170
M9171
M9172
M9173
M9174
M9175
M9176
M9177
M9178
M9179
M9180
M9181
M9182
M9183
M9184
M9185
M9186
M9187
M9188
M9189
M9190
M9191
M9192
M9193
M9194
M9195
M9196
M9197
M9198
M9199
M9200
M9201
M9202
M9203
M9204
M9205
M9206
M9207
M9208
M9209
M9210
M9211
M9212
M9213
M9214
M9215
M9216
M9217
M9218
M9219
M9220
M9221
M9222
M9223
M9224
M9225
M9226
M9227
M9228
M9229
M9230
M9231
M9232
M9233
M9234
M9235
M9236
M9237
M9238
M9239
M9240
M9241
M9242
M9243
M9244
M9245
M9246
M9247
M9248
M9249
M9250
M9251
M9252
M9253
M9254
M9255
M9256
M9257
M9258
M9259
M9260
M9261
M9262
M9263
M9264
M9265
M9266
M9267
M9268
M9269
M9270
M9271
M9272
M9273
M9274
M9275
M9276
M9277
M9278
M9279
M9280
M9281
M9282
M9283
M9284
M9285
M9286
M9287
M9288
M9289
M9290
M9291
M9292
M9293
M929

In [215]:
#drop columns  0
col5 = [3267, 3266, 3265, 3264, 3263, 3262, 3261, 3260, 3259, 0, 2,3,4,5,6]

diadf2= diadf.drop(diadf.columns[col5],axis=1)
diadf2.head()

,SUBJECT_ID,001,002,003,004,005,006,007,008,009,...,M9961,M9962,M9963,M9964,M9965,M9966,M9967,M9968,M9969,M9970
0,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,21,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [219]:
diadf2.shape

(10318, 3253)

In [216]:
diabetesm= pd.merge(matrix2, diadf2, how='inner', on='SUBJECT_ID')

In [217]:
diabetesm.head()

,SUBJECT_ID,LOS,HOSPITALIZATION,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,...,M9961,M9962,M9963,M9964,M9965,M9966,M9967,M9968,M9969,M9970
0,13,3.6660,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,18,1.2885,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20,1.0508,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,21,14.2664,2,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,24,0.5124,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [220]:
diabetesm.shape

(10310, 3314)

In [222]:
diabetesm.loc[diabetesm['LOS']=='0']

/Users/kathy908000/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,SUBJECT_ID,LOS,HOSPITALIZATION,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,...,M9961,M9962,M9963,M9964,M9965,M9966,M9967,M9968,M9969,M9970


In [223]:
diabetesm= diabetesm.sort_values(by=['LOS'])
diabetesm.head()

,SUBJECT_ID,LOS,HOSPITALIZATION,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,...,M9961,M9962,M9963,M9964,M9965,M9966,M9967,M9968,M9969,M9970
5366,29285,0.0006,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4760,26509,0.0164,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1256,6963,0.0281,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
32,235,0.0394,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3751,20960,0.0431,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [225]:
diabetesm= diabetesm.sort_values(by=['LOS'], ascending=False)
diabetesm.head()

,SUBJECT_ID,LOS,HOSPITALIZATION,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,...,M9961,M9962,M9963,M9964,M9965,M9966,M9967,M9968,M9969,M9970
3811,21280,207.1306,8,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3571,19911,152.7318,3,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
8793,78076,134.1599,5,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4102,22818,130.2998,2,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4763,26519,128.9605,4,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [226]:

#diabetesm.to_csv(r'/Users/kathy908000/github/Health-Care-Research/diabetesmatrix.csv', index = False)

In [227]:
diabetesm.shape

(10310, 3314)

In [228]:
for col in diabetesm.columns: 
    print(col) 

SUBJECT_ID
LOS
HOSPITALIZATION
ELECTIVE
EMERGENCY
NEWBORN
URGENT
Government
Medicaid
Medicare
Private
Self Pay
DIVORCED
LIFE PARTNER
MARRIED
NAN
SEPARATED
SINGLE
UNKNOWN
WIDOWED
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
GENDER
001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049
050
051
052
053
054
055
056
057
058
059
060
061
062
063
064
065
066
067
068
069
070
071
072
073
074
075
076
077
078
079
080
081
082
083
084
085
086
087
088
089
090
091
092
093
094
095
096
097
098
099
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
17

M9584
M9585
M9586
M9587
M9588
M9589
M9590
M9591
M9592
M9593
M9594
M9595
M9596
M9597
M9598
M9599
M9600
M9601
M9602
M9603
M9604
M9605
M9606
M9607
M9608
M9609
M9610
M9611
M9612
M9613
M9614
M9615
M9616
M9617
M9618
M9619
M9620
M9621
M9622
M9623
M9624
M9625
M9626
M9627
M9628
M9629
M9630
M9631
M9632
M9633
M9634
M9635
M9636
M9637
M9638
M9639
M9640
M9641
M9642
M9643
M9644
M9645
M9646
M9647
M9648
M9649
M9650
M9651
M9652
M9653
M9654
M9655
M9656
M9657
M9658
M9659
M9660
M9661
M9662
M9663
M9664
M9665
M9666
M9667
M9668
M9669
M9670
M9671
M9672
M9673
M9674
M9675
M9676
M9677
M9678
M9679
M9680
M9681
M9682
M9683
M9684
M9685
M9686
M9687
M9688
M9689
M9690
M9691
M9692
M9693
M9694
M9695
M9696
M9697
M9698
M9699
M9700
M9701
M9702
M9703
M9704
M9705
M9706
M9707
M9708
M9709
M9710
M9711
M9712
M9713
M9714
M9715
M9716
M9717
M9718
M9719
M9720
M9721
M9722
M9723
M9724
M9725
M9726
M9727
M9728
M9729
M9730
M9731
M9732
M9733
M9734
M9735
M9736
M9737
M9738
M9739
M9740
M9741
M9742
M9743
M9744
M9745
M9746
M9747
M9748
M9749
M975

In [ ]:
#icd_grouped.to_csv(r'/Users/kathy908000/github/Health-Care-Research/icd_grouped.csv', index = False)

In [703]:
data2.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [704]:
# multiple rows for each patient? yes
dup = data2[data2.duplicated(['SUBJECT_ID'])]
dup.tail(30)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
58851,57780,96260,110058,2196-08-24 17:05:00,2196-10-03 14:25:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Private,ENGL,PROTESTANT QUAKER,SINGLE,BLACK/AFRICAN AMERICAN,NaN,NaN,STEVENS JOHNSON SYNDROME,0,1
58853,55962,90676,103021,2143-05-28 07:15:00,2143-06-05 13:30:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,Medicare,ENGL,JEWISH,MARRIED,WHITE,NaN,NaN,DRUG REFRACTORY ATRIAL FIBRILLATION\PULMONARY ...,0,1
58858,55967,90688,107472,2153-10-09 08:10:00,2153-10-12 17:31:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2153-10-09 05:16:00,2153-10-09 09:18:00,PNEUMONIA,0,1
58859,55968,90688,127169,2154-05-16 07:15:00,2154-05-16 16:15:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,NaN,NaN,OSTEROARTHRITIS OF LEFT WRIST/SDA,0,0
58860,55969,90688,112686,2154-11-09 11:29:00,2154-11-13 17:30:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2154-11-09 05:33:00,2154-11-09 12:50:00,SEPSIS,0,1
58865,55974,90699,174425,2201-02-24 19:33:00,2201-03-02 16:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,JEWISH,MARRIED,WHITE,2201-02-24 17:36:00,2201-02-24 20:22:00,HYDROCEPHALUS,0,1
58872,56884,93535,121562,2188-08-05 20:27:00,2188-08-07 16:48:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,JEWISH,SINGLE,WHITE,2188-08-05 17:56:00,2188-08-05 18:06:00,ASTHMA;COPD EXACERBATION,0,1
58878,56890,93560,140159,2156-06-11 07:30:00,2156-06-22 16:25:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,Private,ENGL,CATHOLIC,SINGLE,WHITE,NaN,NaN,"TRAUMATIC BRAIN HEMORRHAGE, LEFT /SDA",0,1
58886,57479,95372,171027,2123-04-22 17:14:00,2123-04-29 17:09:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,SPAN,OTHER,MARRIED,HISPANIC OR LATINO,2123-04-22 13:17:00,2123-04-22 19:05:00,ANEMIA/THROMBOCYTOPNEIA/FEVER,0,1
58887,57480,95372,128486,2123-05-16 12:20:00,2123-05-18 13:50:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,SPAN,OTHER,MARRIED,HISPANIC OR LATINO,2123-05-16 07:02:00,2123-05-16 14:54:00,WEAKNESS,0,1


In [705]:
#merged2.iloc[40000:40050]
data2.loc[data2['SUBJECT_ID']==23]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1


In [706]:
data2.shape

(58976, 19)

In [8]:
# duplicate rows?, no
nodup = data2.drop_duplicates(subset=None, keep='first', inplace=False)
nodup.shape

(58976, 19)

In [9]:
data2 = data2.astype ({'ROW_ID': str }) 

In [10]:
#can'get group by with row ids, type error
data6 = data2.groupby('SUBJECT_ID').agg({'ROW_ID' : lambda x: ', '.join(x)})
data6.head(20)

,ROW_ID
SUBJECT_ID,
2,1
3,2
4,3
5,4
6,5
7,6
8,7
9,8
10,9


In [11]:
data3 = data2.drop_duplicates(subset='SUBJECT_ID', keep='first', inplace=False)
data3.shape


(46520, 19)

In [12]:
#Matrix Description
#ethinicty doesn't change, same patient duplicate droped, kept ethnicity in the top row 


#data3 = data2.drop_duplicates(subset='ETHNICITY' , keep='first', inplace=False)
#data3 = data2.groupby('SUBJECT_ID').agg({'ETHNICITY' : lambda x: ', '.join(x)})
#return first value for ethnicity not white twice 

data3 = data3[['SUBJECT_ID','ETHNICITY' ]]

data3.head(30)



,SUBJECT_ID,ETHNICITY
0,22,WHITE
1,23,WHITE
3,24,WHITE
4,25,WHITE
5,26,UNKNOWN/NOT SPECIFIED
6,27,WHITE
7,28,WHITE
8,30,UNKNOWN/NOT SPECIFIED
9,31,WHITE
10,32,WHITE


In [13]:
## group by each colummn and join? 

In [14]:
data4 = data2.groupby('SUBJECT_ID').agg({'INSURANCE' : lambda x: ', '.join(x)})
data4.head()

,INSURANCE
SUBJECT_ID,
2,Private
3,Medicare
4,Private
5,Private
6,Medicare


In [15]:
data5 = data2.groupby('SUBJECT_ID').agg({'ADMISSION_TYPE' : lambda x: ', '.join(x)})
data5.head()

,ADMISSION_TYPE
SUBJECT_ID,
2,NEWBORN
3,EMERGENCY
4,EMERGENCY
5,NEWBORN
6,ELECTIVE


In [16]:
demodata= pd.merge(data3, data4, how='inner', on='SUBJECT_ID')
demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE
0,22,WHITE,Private
1,23,WHITE,"Medicare, Medicare"
2,24,WHITE,Private
3,25,WHITE,Private
4,26,UNKNOWN/NOT SPECIFIED,Medicare


In [17]:
demodata= pd.merge(demodata, data5, how='inner', on='SUBJECT_ID')
demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE
0,22,WHITE,Private,EMERGENCY
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY"
2,24,WHITE,Private,EMERGENCY
3,25,WHITE,Private,EMERGENCY
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY


In [18]:
demodata= pd.merge(demodata, data6, how='inner', on='SUBJECT_ID')
demodata.head(30)

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID
0,22,WHITE,Private,EMERGENCY,21
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23"
2,24,WHITE,Private,EMERGENCY,24
3,25,WHITE,Private,EMERGENCY,25
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26
5,27,WHITE,Private,NEWBORN,27
6,28,WHITE,Medicare,ELECTIVE,28
7,30,UNKNOWN/NOT SPECIFIED,Medicare,URGENT,29
8,31,WHITE,Medicare,EMERGENCY,30
9,32,WHITE,Medicaid,ELECTIVE,31


In [19]:
data2 = data2.astype({'DIAGNOSIS': str })     # ['group2'].astype(str)
#data = data.astype ({'ICD9_CODE': str })

In [20]:
data7 = data2.groupby('SUBJECT_ID').agg({'DIAGNOSIS' : lambda x: ', '.join(x)})
data7.head(40)

,DIAGNOSIS
SUBJECT_ID,
2,NEWBORN
3,HYPOTENSION
4,"FEVER,DEHYDRATION,FAILURE TO THRIVE"
5,NEWBORN
6,CHRONIC RENAL FAILURE/SDA
7,NEWBORN
8,NEWBORN
9,HEMORRHAGIC CVA
10,NEWBORN


In [21]:
demodata= pd.merge(demodata, data7, how='inner', on='SUBJECT_ID')
demodata.head(30)

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH
5,27,WHITE,Private,NEWBORN,27,NEWBORN
6,28,WHITE,Medicare,ELECTIVE,28,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...
7,30,UNKNOWN/NOT SPECIFIED,Medicare,URGENT,29,UNSTABLE ANGINA\CATH
8,31,WHITE,Medicare,EMERGENCY,30,STATUS EPILEPTICUS
9,32,WHITE,Medicaid,ELECTIVE,31,TRACHEAL STENOSIS/SDA


In [22]:
demodata.loc[demodata['SUBJECT_ID']==36]

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS
13,36,WHITE,"Medicare, Medicare, Medicare","EMERGENCY, EMERGENCY, ELECTIVE","36, 37, 38",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...


In [23]:
data7 = data2.groupby('SUBJECT_ID').agg({'ADMISSION_LOCATION' : lambda x: ', '.join(x)})
#data7.head()
demodata= pd.merge(demodata, data7, how='inner', on='SUBJECT_ID')
demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE,EMERGENCY ROOM ADMIT
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,"PHYS REFERRAL/NORMAL DELI, TRANSFER FROM HOSP/..."
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION,TRANSFER FROM HOSP/EXTRAM
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH,TRANSFER FROM HOSP/EXTRAM


In [24]:
data8 = data2.groupby('SUBJECT_ID').agg({'DISCHARGE_LOCATION' : lambda x: ', '.join(x)})
#data8.head()
demodata= pd.merge(demodata, data8, how='inner', on='SUBJECT_ID')
demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION,DISCHARGE_LOCATION
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,"PHYS REFERRAL/NORMAL DELI, TRANSFER FROM HOSP/...","HOME HEALTH CARE, HOME HEALTH CARE"
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION,TRANSFER FROM HOSP/EXTRAM,HOME
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT,HOME
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH,TRANSFER FROM HOSP/EXTRAM,HOME


In [25]:
data2.dtypes

ROW_ID                  object
SUBJECT_ID               int64
HADM_ID                  int64
ADMITTIME               object
DISCHTIME               object
DEATHTIME               object
ADMISSION_TYPE          object
ADMISSION_LOCATION      object
DISCHARGE_LOCATION      object
INSURANCE               object
LANGUAGE                object
RELIGION                object
MARITAL_STATUS          object
ETHNICITY               object
EDREGTIME               object
EDOUTTIME               object
DIAGNOSIS               object
HOSPITAL_EXPIRE_FLAG     int64
HAS_CHARTEVENTS_DATA     int64
dtype: object

In [26]:
data2 = data2.astype ({'MARITAL_STATUS': str }) 

In [27]:
data9 = data2.groupby('SUBJECT_ID').agg({'MARITAL_STATUS' : lambda x: ', '.join(x)})
#data8.head()
demodata= pd.merge(demodata, data9, how='inner', on='SUBJECT_ID')
demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION,DISCHARGE_LOCATION,MARITAL_STATUS
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,MARRIED
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,"PHYS REFERRAL/NORMAL DELI, TRANSFER FROM HOSP/...","HOME HEALTH CARE, HOME HEALTH CARE","MARRIED, MARRIED"
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT,HOME,MARRIED
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE


In [28]:
data2 = data2.astype ({'HADM_ID': str }) 

In [29]:
data10 = data2.groupby('SUBJECT_ID').agg({'HADM_ID' : lambda x: ', '.join(x)})
#data7.head()
demodata= pd.merge(demodata, data10, how='inner', on='SUBJECT_ID')
demodata.tail(50)

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION,DISCHARGE_LOCATION,MARITAL_STATUS,HADM_ID
46470,98636,BLACK/AFRICAN AMERICAN,Medicaid,EMERGENCY,58537,ASTHMA;CHRONIC OBST PULM DISEASE,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,SINGLE,143528
46471,98639,WHITE,Medicare,ELECTIVE,58538,CONGESTIVE HEART FAILURE\Transcatheter Aortic ...,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,MARRIED,167385
46472,98640,WHITE,Private,EMERGENCY,58539,INTRACRANIAL HEMORRHAGE,CLINIC REFERRAL/PREMATURE,HOME,SINGLE,104073
46473,98642,WHITE,Medicare,EMERGENCY,58540,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,SINGLE,162213
46474,98643,PATIENT DECLINED TO ANSWER,Private,ELECTIVE,58541,ATRIAL FIBILLATION\BILATERAL THORACOSCOPIC MIN...,PHYS REFERRAL/NORMAL DELI,HOME,MARRIED,177932
46475,98644,BLACK/AFRICAN AMERICAN,Medicare,EMERGENCY,58542,ACUTE RENAL FAILURE,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,MARRIED,109643
46476,98647,WHITE,Medicare,EMERGENCY,58543,LEFT FEMUR FRACTURE,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,SINGLE,167391
46477,98649,ASIAN - ASIAN INDIAN,Medicaid,EMERGENCY,58544,PNEUMONIA,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,WIDOWED,110990
46478,98656,WHITE,Medicare,EMERGENCY,58545,ABDOMINAL PAIN,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,WIDOWED,138450
46479,98664,WHITE,Private,EMERGENCY,58546,CHEST PAIN,EMERGENCY ROOM ADMIT,SNF,SEPARATED,150793


In [30]:
# match with ICD9 code using HADMID? 
demodata.shape

(46520, 10)

# Check if Regex to find row works

In [31]:
sample = demodata.loc[demodata['SUBJECT_ID']==34]
sample['ROW_ID']

11    33, 34
Name: ROW_ID, dtype: object

In [33]:
#doesn't work
#demodata.loc[demodata['ROW_ID']==[33,34]]


In [37]:
#change groupby to make 1 and 0 regex work, no comma, a string
sample2 = data2.groupby('SUBJECT_ID').agg({'ADMISSION_TYPE' : lambda x: ''.join(x)})
data5.head(30)

,ADMISSION_TYPE
SUBJECT_ID,
2,NEWBORN
3,EMERGENCY
4,EMERGENCY
5,NEWBORN
6,ELECTIVE
7,NEWBORN
8,NEWBORN
9,EMERGENCY
10,NEWBORN


In [33]:
#sample2.loc[sample2['ADMISSION_TYPE'].str.contains('NEW'), 'ADMISSION_TYPE'] = 1
#sample2.head(10)

#rows = sample2[sample2['ADMISSION_TYPE'].str.contains('NEW')]
#dataframe with selected rows 
#rows['ADMISSION_TYPE'].iloc['']

#df.b.str.contains('^f')

#groups to int

#only numeric columns datamap['group'] = datamap['group'].str.replace('V','10',regex=True) datamap['group'] = datamap['group'].str.replace('E','11',regex=True)

#datamap['group'] = datamap['group'].astype(np.int) datamap['group'] = datamap['group'].astype(np.float) datamap.head(30)

In [49]:
#with random dataframe
train = pd.DataFrame({'Cabin':[10,20,30,40,50], 'Pclass':[1,2,3,1,2]})
print (train)

   Cabin  Pclass
0     10       1
1     20       2
2     30       3
3     40       1
4     50       2


In [50]:
train.loc[train['Pclass'] == 1, 'Pclass'] = 2
print(train)

   Cabin  Pclass
0     10       2
1     20       2
2     30       3
3     40       2
4     50       2


# Age

In [161]:
agedata= pd.read_csv('PATIENTS.csv')
agedata.shape

(46520, 8)

In [162]:
#admission and patient.csv date of admission - doB

# LabelEncoding

In [163]:
le = LabelEncoder() 

In [164]:
demodata['ETHNICITYc']= le.fit_transform(demodata['ETHNICITY']) 

demodata.head()

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION,DISCHARGE_LOCATION,MARITAL_STATUS,HADM_ID,ETHNICITYc
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,MARRIED,165315,36
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,"PHYS REFERRAL/NORMAL DELI, TRANSFER FROM HOSP/...","HOME HEALTH CARE, HOME HEALTH CARE","MARRIED, MARRIED","152223, 124321",36
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE,161859,36
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT,HOME,MARRIED,129635,36
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE,197661,35


In [167]:
demodata['INSURANCEc']= le.fit_transform(demodata['INSURANCE']) 
demodata.head(10)

,SUBJECT_ID,ETHNICITY,INSURANCE,ADMISSION_TYPE,ROW_ID,DIAGNOSIS,ADMISSION_LOCATION,DISCHARGE_LOCATION,MARITAL_STATUS,HADM_ID,ETHNICITYc,INSURANCEc
0,22,WHITE,Private,EMERGENCY,21,BENZODIAZEPINE OVERDOSE,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,MARRIED,165315,36,154
1,23,WHITE,"Medicare, Medicare","ELECTIVE, EMERGENCY","22, 23",CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,"PHYS REFERRAL/NORMAL DELI, TRANSFER FROM HOSP/...","HOME HEALTH CARE, HOME HEALTH CARE","MARRIED, MARRIED","152223, 124321",36,105
2,24,WHITE,Private,EMERGENCY,24,INTERIOR MYOCARDIAL INFARCTION,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE,161859,36,154
3,25,WHITE,Private,EMERGENCY,25,ACUTE CORONARY SYNDROME,EMERGENCY ROOM ADMIT,HOME,MARRIED,129635,36,154
4,26,UNKNOWN/NOT SPECIFIED,Medicare,EMERGENCY,26,V-TACH,TRANSFER FROM HOSP/EXTRAM,HOME,SINGLE,197661,35,98
5,27,WHITE,Private,NEWBORN,27,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,nan,134931,36,154
6,28,WHITE,Medicare,ELECTIVE,28,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,MARRIED,162569,36,98
7,30,UNKNOWN/NOT SPECIFIED,Medicare,URGENT,29,UNSTABLE ANGINA\CATH,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,MARRIED,104557,35,98
8,31,WHITE,Medicare,EMERGENCY,30,STATUS EPILEPTICUS,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,MARRIED,128652,36,98
9,32,WHITE,Medicaid,ELECTIVE,31,TRACHEAL STENOSIS/SDA,PHYS REFERRAL/NORMAL DELI,HOME,nan,175413,36,35


# Diagnosis

In [36]:
data= pd.read_csv('group19gender.csv')
data.shape # doesn't have empty rows
#data.head()
#data['SUBJECT_ID'].head(10)


/Users/kathy908000/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(634709, 35)

In [37]:
data.loc[6:11].head(10)

,ROW_ID_x,ICD9_CODE,SHORT_TITLE_x,LONG_TITLE_x,group,group2,lv.1_code,lv.2_code,lv.1_group,lv.2_group,...,bg10,bg11,bg12,bg13,bg14,bg15,bg16,bg17,bg18,bg19
6,252.0,41,Shigella flexneri,Shigella flexneri,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
7,252.0,41,Shigella flexneri,Shigella flexneri,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
8,252.0,41,Shigella flexneri,Shigella flexneri,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
9,255.0,48,Shigella infection NEC,Other specified shigella infections,0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
10,256.0,49,Shigellosis NOS,"Shigellosis, unspecified",0,1.0,001-009,4.0,INTESTINAL INFECTIOUS DISEASES,Shigellosis,...,0,0,0,0,0,0,0,0,0,0
11,258.0,51,Botulism food poisoning,Botulism food poisoning,0,1.0,001-009,5.0,INTESTINAL INFECTIOUS DISEASES,Other food poisoning (bacterial),...,0,0,0,0,0,0,0,0,0,0


In [38]:
data = data.drop_duplicates(subset=None, keep='first', inplace=False)
data.shape

(634649, 35)

# Each Row Patient

In [39]:
#data['group2'] = data['group2'].astype(str)
data = data.astype({'group2': str })     # ['group2'].astype(str)
data = data.astype ({'ICD9_CODE': str }) 

In [40]:
data.dtypes

ROW_ID_x         float64
ICD9_CODE         object
SHORT_TITLE_x     object
LONG_TITLE_x      object
group             object
group2            object
lv.1_code         object
lv.2_code        float64
lv.1_group        object
lv.2_group        object
SUBJECT_ID       float64
HADM_ID          float64
GENDER             int64
DOB               object
DOD              float64
DOD_HOSP          object
bg1                int64
bg2                int64
bg3                int64
bg4                int64
bg5                int64
bg6                int64
bg7                int64
bg8                int64
bg9                int64
bg10               int64
bg11               int64
bg12               int64
bg13               int64
bg14               int64
bg15               int64
bg16               int64
bg17               int64
bg18               int64
bg19               int64
dtype: object

In [41]:
#group by groupconcacat
#data = data.groupby('SUBJECT_ID').agg({'group2' : lambda x: ', '.join(x)})
#data = data.groupby('SUBJECT_ID').agg({'group2' : lambda x: ', '.join(x)}, {'ICD9_CODE' :lambda x: ', '.join(x) })

#data = data.groupby('SUBJECT_ID').agg({'group2' : lambda x: ', '.join(x), 'ICD9_CODE' :lambda x: ', '.join(x) })
#data.head()

In [42]:
pdata = data.groupby('SUBJECT_ID')['ICD9_CODE'].apply(lambda group_series: group_series.tolist()).reset_index()
pdata.head(10)


,SUBJECT_ID,ICD9_CODE
0,2.0,"[V053, V290, V3001]"
1,3.0,"[389, 2639, 41071, 4254, 4275, 4280, 5849, 682..."
2,4.0,"[4111, 42, 1363, 2763, 5715, 7994, 7907, E9317..."
3,5.0,"[V053, V290, V3000]"
4,6.0,"[2753, 2767, 2859, 40391, 9972, V1582]"
5,7.0,"[V053, V290, V3001]"
6,8.0,"[7706, 7746, V053, V290, V3001, V502]"
7,9.0,"[4019, 4280, 431, 5070, 5849]"
8,10.0,"[76525, 76515, 7742, V290, V3000]"
9,11.0,[1913]


In [43]:
#to group based on ICD9-code, one hot encoding each category first 2 digits

#pdata['2dgroup'] = list(map(lambda x: x[0:2], list(pdata['ICD9_CODE'])))
#pdata.head()

# find all diagnoses of single patient, make a list, make new dataframe

In [44]:
icd9 = pdata['ICD9_CODE']
icd9

0                                      [V053, V290, V3001]
1        [389, 2639, 41071, 4254, 4275, 4280, 5849, 682...
2        [4111, 42, 1363, 2763, 5715, 7994, 7907, E9317...
3                                      [V053, V290, V3000]
4                   [2753, 2767, 2859, 40391, 9972, V1582]
                               ...                        
46512    [389, 23871, 2449, 2724, 2859, 48241, 4870, 51...
46513    [389, 25000, 27651, 40291, 4280, 5538, 5570, 5...
46514    [2721, 2851, 3004, 4019, 53081, 56881, 5699, 5...
46515    [25000, 2724, 2851, 41401, 4241, 4280, 42833, ...
46516                     [25000, 4019, 4139, 75612, 7861]
Name: ICD9_CODE, Length: 46517, dtype: object

digit = []
row = []
a = []

for i in range(634650):
    for j in icd9.iloc[i]:
        digit = j[0:2]
        row.append(digit)
    a.append(row)

a
    
